# Pruebas con barrido de frecuencia

In [1]:
import threading
import queue
import time
import numpy as np
import sounddevice as sd
import serial
import pickle
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import matplotlib.pyplot as plt
from ordered_set import OrderedSet

In [2]:
import sys
sys.path.append('..')
# Custom imports
from analysis_utils import DataContainer
from audio_constants import ref_volume, freq_response_waves, freq_sweep_waves, freq_weighting_waves, dB_to_scalar

In [3]:
ESP_COM_PORT = "COM22"
ESP_BAUDRATE = 115200

SLM_COM_PORT = "COM23"
SLM_BAUDRATE = 115200

In [4]:
# Get the default audio playback device
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)

# Create a volume control interface
volume_controller = cast(interface, POINTER(IAudioEndpointVolume))

In [5]:
# Variables for status syncronization between threads
current_freq = 0
current_volume = 0.0

In [6]:
# Function to receive serial data and store it
def receive_esp_serial_data(data, queue):
    ser = serial.Serial(ESP_COM_PORT, ESP_BAUDRATE)
    while not finish_flag:
        received_data = ser.readline().decode().strip()
        received_data = received_data.split(" ")[0]
        try:
            received_data = float(received_data)
        except ValueError:
            received_data = float(received_data.split(".")[0])
        try:
            if log_flag:
                data[current_freq][current_volume] = np.append(data[current_freq][current_volume], received_data)
        except KeyError:
            pass
    ser.close()
    queue.put(data)

# Function to receive serial data and store it
def receive_slm_serial_data(data, queue):
    ser = serial.Serial(SLM_COM_PORT, SLM_BAUDRATE)
    while not finish_flag:
        received_data = ser.readline().decode().strip()
        received_data = received_data.split(" ")[0]
        try:
            received_data = float(received_data)
        except ValueError:
            received_data = float(received_data.split(".")[0])
        try:
            if log_flag:
                data[current_freq][current_volume] = np.append(data[current_freq][current_volume], received_data)
        except KeyError:
            pass
    ser.close()
    queue.put(data)

# Function to play audio sine waves
def play_test_waves(test_waves):
    global current_freq, current_volume, log_flag, freq_flag, finish_flag

    duration = 2.5 if TIME_PERIOD == 0.125 else 20
    sample_rate = 88200
    t = np.arange(0, duration, 1 / sample_rate)

    finish_flag = False

    for freq in test_waves["frequencies"]:
        current_freq = freq
        sine_wave = np.sin(2 * np.pi * freq * t)

        for n, volume in enumerate(test_waves["volumes"]):
            current_volume = volume
            print(f"now playing: {volume} volume @ {freq} Hz")
            log_flag = True
            if not freq_flag:
                freq_flag = True

            volume_controller.SetMasterVolumeLevelScalar(dB_to_scalar[volume], None)

            # Wait before playing the sine wave
            time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

            sd.play(sine_wave, sample_rate, blocking=True)

            # Wait after playback is done
            time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

            log_flag = False
            if n == (len(test_waves["volumes"]) - 1):
                freq_flag = False

    # Send a signal to the other thread to terminate
    finish_flag = True

In [7]:
# Create formatted containers to store received serial data
freq_response_data = DataContainer(freq_response_waves)
freq_response_data_ref = DataContainer(freq_response_waves)
freq_sweep_data_F = DataContainer(freq_sweep_waves)
freq_sweep_data_F_ref = DataContainer(freq_sweep_waves)
freq_sweep_data_S = DataContainer(freq_sweep_waves)
freq_sweep_data_S_ref = DataContainer(freq_sweep_waves)
freq_weighting_data = DataContainer(freq_weighting_waves)
freq_weighting_data_ref = DataContainer(freq_weighting_waves)

esp_queue = queue.Queue()
slm_queue = queue.Queue()

# Flag to signal termination
finish_flag = False
# Flag to signal when to log measurements
log_flag = False

## Ejecutar pruebas

### Respuesta de frecuencia y nivel

In [ ]:
TIME_PERIOD = 0.125

# *** Frequency response ***

# Create the serial data receiving thread
esp_serial_thread = threading.Thread(target=receive_esp_serial_data, args=(freq_response_data,esp_queue), daemon=True)
#slm_serial_thread = threading.Thread(target=receive_slm_serial_data, args=(freq_response_data_ref,slm_queue), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_waves, args=(freq_response_waves,), daemon=True)

# Start serial thread
esp_serial_thread.start()
#slm_serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
esp_serial_thread.join()
#slm_serial_thread.join()
audio_thread.join()

# Reset the flag for next test
finish_flag = False

# Retrieve data from the thread
freq_response_data = esp_queue.get()
#freq_response_data_ref = slm_queue.get()

### Barrido de frecuencia y nivel con periodo F

In [ ]:
TIME_PERIOD = 0.125

# Create the serial data receiving thread
esp_serial_thread = threading.Thread(target=receive_esp_serial_data, args=(freq_sweep_data_F,esp_queue), daemon=True)
#slm_serial_thread = threading.Thread(target=receive_slm_serial_data, args=(freq_sweep_data_F_ref,slm_queue), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_waves, args=(freq_sweep_waves,), daemon=True)

# Start serial thread
esp_serial_thread.start()
#slm_serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
esp_serial_thread.join()
#slm_serial_thread.join()
audio_thread.join()

# Reset the flag for next test
finish_flag = False

# Retrieve data from the thread
freq_sweep_data_F = esp_queue.get()
#freq_sweep_data_F_ref = slm_queue.get()

### Barrido de frecuencia y nivel con periodo S

In [ ]:
TIME_PERIOD = 1.0

# Create the serial data receiving thread
esp_serial_thread = threading.Thread(target=receive_esp_serial_data, args=(freq_sweep_data_S,esp_queue), daemon=True)
#slm_serial_thread = threading.Thread(target=receive_slm_serial_data, args=(freq_sweep_data_S_ref,slm_queue), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_waves, args=(freq_sweep_waves,), daemon=True)

# Start serial thread
esp_serial_thread.start()
#slm_serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
esp_serial_thread.join()
#slm_serial_thread.join()
audio_thread.join()

# Reset the flag for next test
finish_flag = False

# Retrieve data from the thread
freq_sweep_data_S = esp_queue.get()
#freq_sweep_data_S_ref = slm_queue.get()

### Ponderación de frecuencia

In [ ]:
TIME_PERIOD = 0.125

# Create the serial data receiving thread
esp_serial_thread = threading.Thread(target=receive_esp_serial_data, args=(freq_weighting_data,esp_queue), daemon=True)
#slm_serial_thread = threading.Thread(target=receive_slm_serial_data, args=(freq_weighting_data_ref,slm_queue), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_waves, args=(freq_weighting_waves,), daemon=True)

# Start serial thread
esp_serial_thread.start()
#slm_serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
esp_serial_thread.join()
#slm_serial_thread.join()
audio_thread.join()

# Reset the flag for next test
finish_flag = False

# Retrieve data from the thread
freq_weighting_data = esp_queue.get()
#freq_weighting_data_ref = slm_queue.get()

## Guardar datos

In [69]:
with open("../results/sweep_tests/freq_response.pkl", 'wb') as file:
    pickle.dump(freq_response_data, file)

with open("../results/sweep_tests/freq_sweep_f.pkl", 'wb') as file:
    pickle.dump(freq_sweep_data_F, file)

with open("../results/sweep_tests/freq_sweep_s.pkl", 'wb') as file:
    pickle.dump(freq_sweep_data_S, file)

with open("../results/sweep_tests/freq_wting.pkl", 'wb') as file:
    pickle.dump(freq_weighting_data, file)

In [71]:
with open("../results/sweep_tests/freq_response_ref.pkl", 'wb') as file:
    pickle.dump(freq_response_data_ref, file)

with open("../results/sweep_tests/freq_sweep_f_ref.pkl", 'wb') as file:
    pickle.dump(freq_sweep_data_F_ref, file)

with open("../results/sweep_tests/freq_sweep_s_ref.pkl", 'wb') as file:
    pickle.dump(freq_sweep_data_S_ref, file)

with open("../results/sweep_tests/freq_wting_ref.pkl", 'wb') as file:
    pickle.dump(freq_weighting_data_ref, file)